In [ ]:
from dataclasses import dataclass
from observer import CalibratedObserver, TrackedObject, CalibratedCaptureConfiguration, Transition, distanceFormula
import dma.MechaCombat as mc
from dma.MechaCombat import GameEvent, GameState, Skill, Mech
QuantumSystem = mc.QuantumSystem
qs = QuantumSystem

In [ ]:
@dataclass
class HarmonyObject(TrackedObject):
    oid: str = None
    objectType: str = None
    objectSubType: str = None
    objectKwargs: dict = None

    objectFactories = {"Unit": mc.MechFactories, "Structure": mc.StructureFactories}
    
    def __post_init__(self):
        super().__post_init__()
        print(f"Creating {self.objectType} -- {self.objectSubType} -- {self.objectKwargs}")
        self.objectKwargs = {key.lower(): value for key, value in self.objectKwargs.items()}
        try:
            factory = self.objectFactories[self.objectType][self.objectSubType]
        except KeyError as ke:
            raise Exception(f"Unrecognized object type and subtype - {self.objectType}-{self.objectSubType}")
        self.object = factory(self.oid, faction="Unaligned", **self.objectKwargs)
        GameState.addObject(self.object.entity, self.objectType)

    @classmethod
    def from_tracked_object(cls,
                            trackedObject: TrackedObject,
                            objectType: str,
                            objectSubType: str,
                            objectKwargs: dict):
        # if not qs.entity_exists(trackedObject.oid):
        print(f"Creating {objectType} object from {trackedObject}")
        return cls(
            changeSet=trackedObject.changeSet,
            oid=trackedObject.oid,
            objectType=objectType,
            objectSubType=objectSubType,
            objectKwargs=objectKwargs)
        
    def previousVersion(self):
        return TrackedObject({camName: change.lastChange if change is not None else None for camName, change in self.changeSet.items()})

    def rename(self, newName):
        qs.rename_entity(self.oid, newName)
        self.oid = newName

In [ ]:
@dataclass
class ObjectAction:
    actor: HarmonyObject
    target: HarmonyObject
    result: str = "null"
    
    def __post_init__(self):
        assert self.rsc is not None, f"Object Actions require a RealSpace Converter"
        if self.target is None:
            self.targetDistance = None
            self.targetRange = None
            self.rangeModifier = None
            self.objMovement = None
            self.aMM = None
            self.targetMovement = None
            self.tMM = None
            self.skill = None
            self.targetNumber = None
            self.gameEvent = None
            return
        self.targetDistance = self.rsc.distanceBetweenObjects(self.actor, self.target)
        self.targetRange = "short" if self.targetDistance < 155 else "medium" if self.targetDistance < 610 else "long"
        self.rangeModifier = 0 if "short" else 2 if "medium" else 4

        self.objMovement = self.rsc.trackedObjectLastDistance(self.actor)
        self.aMM = -1 if self.objMovement is None or self.objMovement < 10 else 1
        self.targetMovement = self.rsc.trackedObjectLastDistance(self.target)
        self.tMM = -1 if self.targetMovement is None or self.targetMovement < 10 else 1
        self.skill = Skill(self.actor.oid).terminant
        self.targetNumber = int(self.skill) + self.aMM + self.tMM + 0 + self.rangeModifier
        self.gameEvent = None

    def declare(self):
        self.gameEvent = GameEvent.declareEvent("Attack", "Unaligned", self.actor.oid, self.target.oid, self.result)

    def resolve(self, result):
        self.gameEvent.resolve(result)

In [ ]:
class HarmonyMachine(CalibratedObserver):
    def reset(self):
        super().reset()
        mc.GameState.reset()
        self.GameState = mc.GameState
        ObjectAction.rsc = self.cc.rsc

    def declareEvent(self, eventType: str, eventFaction: str, eventObject: str, eventValue: str, eventResult: str):
        mc.GameState.declareEvent(eventType, eventFaction, eventObject, eventValue, eventResult)
    
    def classifyObject(self, objectId, objectType, objectSubType, objectKwargs):
        trackedObject = self.findObject(objectId)
        memoryIndex = self.memory.index(trackedObject)
        harmonyObject = HarmonyObject.from_tracked_object(
            trackedObject, objectType, objectSubType, objectKwargs)
        mc.XYLocation.set_relationship(harmonyObject.oid, self.cc.rsc.changeSetToRealCenter(harmonyObject))
        self.memory[memoryIndex] = harmonyObject
        return harmonyObject

    def captureRealCoord(self, capture):
        """ movement distance this round """
        return self.changeSetToRealCenter(capture)

    def captureSystemCoord(self, capture):
        return mc.XYLocation(capture.oid)

    def commitChanges(self, objDef, overwrite=True):
        try:
            existingIndex = self.memory.index(objDef.previousVersion())
            print(f"Updating Memory {existingIndex}")
            self.memory[existingIndex].update(objDef, overwrite=overwrite)
        except ValueError:
            print(f"New Memory")
            if GameState.getPhase() != "Add":
                raise Exception("Cannot make new memories outside of Add Phase")
            self.memory.append(objDef)
            existingIndex = len(self.memory) - 1
        self.lastMemory = objDef
        
        self.transitions.append(Transition(
            objDef,
            self.cycleCounter,
            {camName: {"ref": cam.referenceFrame, "fin": cam.mostRecentFrame}
             for camName, cam in self.cc.cameras.items()}))
    
        if isinstance(objDef, HarmonyObject):
            systemCoord = self.captureSystemCoord(objDef)
            realCoord = self.captureRealCord(objDef)
            delta = [d1 - d0 for d0, d1 in zip(systemCoord, realCoord)]
            delta.append(0)  # TODO: get elevation change from terrain
            if mc.Move.can_move(objDef, delta):
                self.declareEvent(
                    eventType="Move",
                    eventFaction=mc.Faction(objDef.oid),
                    eventObject=objDef.oid,
                    eventValue=json.dumps(delta),
                    eventResult=json.dumps(realCoord))
        return objDef

    def takeAction(self, actor, target, result=None):
        return ObjectAction(actor, target, result=result)

    def distanceFromRealPoint(self, harmonyObject: HarmonyObject, realPoint: (float, float)):
        # TODO: object at realpoint? Elevation at realpoint?
        objectLocation = mc.XYLocation(harmonyObject.oid).terminant
        return distanceFormula(objectLocation, realPoint)

    def objectLastDistance(self, harmonyObject: HarmonyObject):
        if mc.GameState.getPhase() == "Move":
            currentLocation = self.cc.rsc.changeSetToRealCenter(harmonyObject)
            return self.distanceFromRealPoint(harmonyObject, currentLocation) * 0.03937008
        else:
            return mc.DistanceMoved(harmonyObject.oid).terminant

    def objectHasMoved(self, harmonyObject: HarmonyObject):
        if self.objectLastDistance(harmonyObject) > 0.3:
            return True
        return False

    def unitsMovedThisRound(self):
        mechs = Mech.active_mechs()
        moved = []
        for m in mechs:
            harmonyObject = self.findObject(m)
            if self.objectHasMoved(harmonyObject):
                moved.append(m)
        return moved

    def objectHasDeclared(self, harmonyObject):
        if len(GameEvent.get_existing_declarations(harmonyObject.oid)) > 0:
            return True
        return False

    def unitsDeclaredThisRound(self):
        mechs = Mech.entities()
        declared = []
        for m in mechs:
            harmonyObject = self.findObject(m)
            if self.objectHasDeclared(harmonyObject):
                declared.append(m)
        return declared

    def objectHasResolved(self, harmonyObject):
        for re in GameEvent.get_resolved_events():
            if harmonyObject.oid == re.GameEventObject.terminant:
                return True
        return False

    def actionsToResolve(self):
        declared_actions = len(app.cm.unitsDeclaredThisRound())
        no_actions = len(GameEvent.get_declared_no_action_events())
        return declared_actions - no_actions

    def commitMovement(self):
        self.passiveMode()
        for entity in mc.Movement.entities():
            obj = self.findObject(entity)
            if self.objectHasMoved(obj):
                mc.Move.move_to(entity=entity, newLocation=[*self.cc.rsc.changeSetToRealCenter(obj), 0])
        mc.GameState.newPhase("Move")

    def resolveRound(self):
        for entity in mc.Movement.entities():
            mc.DistanceMoved.set_relationship(entity, 0)
        mc.GameState.newPhase("Action")
        self.trackMode()

    def units(self):
        return mc.Mech.entities()

    def active_units(self):
        return mc.Mech.active_mechs()

    def obj_destroyed(self, oid: str):
        return mc.UnitDamage.is_destroyed(entity=oid)

    def mech_skill(self, mech_oid: str):
        return mc.Skill(mech_oid).terminant

In [ ]:
if __name__ == "__main__":
    from matplotlib import pyplot as plt
    cc = CalibratedCaptureConfiguration()
    hm = HarmonyMachine(cc)
    hm.cycle()
    plt.imshow(cc.cameras['0'].mostRecentFrame)

In [ ]:
if __name__ == "__main__":
    for i in range(1):
        hm.cycleForChange()
    for i in range(3):
        hm.cycle()
    plt.imshow(hm.getCameraImagesWithChanges()['0'])